In [2]:
from collections import defaultdict

In [3]:
def formatter(string):
    case, event = string.replace("\n","").split(",")
    return case, event

# returns a set with all the activities present in the log
# and a dictionary with each event's trace of events
# where the key of the dict is the caseId and the value
# is an array of events
def process_logs(filename):
    logs = defaultdict(list)
    events = set()
    with open(filename, "r") as logfile:
        for i in logfile.readlines()[1:]:
            case, event = formatter(i)
            logs[case].append(event)
            events.add(event)
    return events, logs
        

In [4]:
# returns a matrix filled with zeros
# with dimension equal to the event_set squared.
def zero_transition_matrix(event_set):
    event_list = list(event_set)
    event_list.sort()
    event_indexes = {event: index for index, event in enumerate(event_list)}
    event_count = len(event_list)
    matrix = [[0] * event_count for i in range(event_count)]
    return event_indexes, matrix

In [6]:
events, logs = process_logs("TDlog.csv")
event_indexes, transition_matrix = zero_transition_matrix(events)
def generate_succession_matrix(events, logs, window=1):
    event_indexes, transition_matrix = zero_transition_matrix(events)
    for log in logs.values():
        last_checkable_index = len(log) - window
        for index in range(0, last_checkable_index):
            predecessor = log[index]
            successor = log[index + window]
            predecessor_index = event_indexes[predecessor]
            successor_index = event_indexes[successor]
            transition_matrix[predecessor_index][successor_index] += 1
    return transition_matrix

succession_matrix = generate_succession_matrix(events, logs,2)
for letter, index in event_indexes.items():
    for letter2, index2 in event_indexes.items():
        count = succession_matrix[index][index2]
        if count > 0:
            print("{}: {} -> {}".format(letter, letter2, count))

A: D -> 250
A: E -> 261
A: F -> 261
A: G -> 228
B: B -> 499
B: M -> 268
B: P -> 243
C: H -> 131
C: I -> 130
C: K -> 104
C: L -> 124
D: D -> 249
D: E -> 250
E: M -> 243
E: P -> 268
F: H -> 130
F: I -> 131
G: L -> 104
H: J -> 131
H: L -> 130
I: J -> 130
I: L -> 131
M: N -> 268
M: O -> 243
N: Q -> 511
O: L -> 511
P: N -> 243
P: O -> 268


In [5]:
events, logs = process_logs("TDlog.csv")
def generate_dependancy_matrix(events, logs):
    succession_matrix = generate_succession_matrix(events, logs, 1)
    event_indexes, dependancy_matrix = zero_transition_matrix(events)
    dimension = len(succession_matrix)
    for row in range(dimension):
        for col in range(dimension):
            direct_succession = succession_matrix[row][col]
            inverse_succession = succession_matrix[col][row]
            if row == col:
                dependancy_matrix[row][col] = round((direct_succession / (direct_succession + 1)), 3)
            else:
                dependancy_matrix[row][col] = round((direct_succession - inverse_succession) / (direct_succession + inverse_succession + 1), 3)
    return dependancy_matrix
dependancy_matrix = generate_dependancy_matrix(events, logs)
for letter, index in event_indexes.items():
    for letter2, index2 in event_indexes.items():
        count = dependancy_matrix[index][index2]
        if count > 0:
            print("{}: {} -> {}".format(letter, letter2, count))

A: B -> 0.998
A: C -> 0.998
B: E -> 0.998
C: F -> 0.996
C: G -> 0.996
E: M -> 0.996
E: P -> 0.996
F: H -> 0.992
F: I -> 0.992
G: K -> 0.99
G: L -> 0.992
H: I -> 0.004
H: J -> 0.992
I: J -> 0.992
J: L -> 0.996
K: L -> 0.99
M: N -> 0.996
M: P -> 0.049
N: O -> 0.998
O: Q -> 0.998
P: N -> 0.996
Q: L -> 0.998


In [6]:
def indexes_to_events(dictionary, transform):
    inverse = { v: k for k, v in transform.items() }
    result = {}
    for key, array in dictionary.items():
        letter = inverse[key]
        letter_array = [inverse[item] for item in array]
        result[letter] = letter_array
    return result

In [7]:
def is_above_thresholds(frecuency, dependancy, thresholds):
    return (frecuency > thresholds["frecuency"] and dependancy > thresholds["dependancy"])

def is_below_thresholds(frecuency, dependancy, thresholds):
    return (frecuency <= thresholds["frecuency"] and dependancy <= thresholds["dependancy"])

def is_above_frecuency_below_dependancy(frecuency, dependancy, thresholds):
    return (frecuency > thresholds["frecuency"] and dependancy <= thresholds["dependancy"])

In [8]:
def edges_above_threshold(edges, thresholds, succession_matrix, dependancy_matrix):
    conditions = {}
    for start, end in edges:
        frecuency = succession_matrix[start][end]
        dependancy = dependancy_matrix[start][end]
        conditions[(start, end)] = is_above_thresholds(frecuency, dependancy, thresholds)
    return conditions

def edges_below_threshold(edges, thresholds, succession_matrix, dependancy_matrix):
    conditions = {}
    for start, end in edges:
        frecuency = succession_matrix[start][end]
        dependancy = dependancy_matrix[start][end]
        conditions[(start, end)] = is_below_thresholds(frecuency, dependancy, thresholds)
    return conditions

def edges_above_frecuency_below_dependancy(edges, thresholds, succession_matrix, dependancy_matrix):
    conditions = {}
    for start, end in edges:
        frecuency = succession_matrix[start][end]
        dependancy = dependancy_matrix[start][end]
        conditions[(start, end)] = is_above_frecuency_below_dependancy(frecuency, dependancy, thresholds)
    return conditions

In [9]:
def find_xor_splits(event_indexes, succession_matrix, dependancy_matrix, thresholds):
    xor_nodes = defaultdict(list)
    for row in event_indexes.values():
        xor_edges = []
        for col in event_indexes.values():
            xor_edges.append([row, col])
        above_threshold = edges_above_threshold(xor_edges, thresholds, succession_matrix, dependancy_matrix)
        for edge, above in above_threshold.items():
            if above:
                xor_nodes[edge[0]].append(edge[1])
    xor_nodes = { k: v for k, v in xor_nodes.items() if len(v) > 1 }
    print("Potential XOR splits (format -> 'start: [ends]'): {}".format(indexes_to_events(xor_nodes, event_indexes)))    
    xor_filtered_nodes = xor_nodes.copy()
    for start_node, successors in xor_nodes.items():
        edges = []
        others = [start_node] + successors        
        for successor in successors:
            for other in others:
                if successor == other:
                    continue
                edges.append([successor, other])
        below_threshold = edges_below_threshold(edges, thresholds, succession_matrix, dependancy_matrix)                
        if not all(condition for condition in below_threshold.values()):
            xor_filtered_nodes.pop(start_node, None)
    print("Discovered XOR splits: {}".format(indexes_to_events(xor_filtered_nodes, event_indexes)))
    return xor_filtered_nodes


In [10]:
def find_xor_joins(event_indexes, succession_matrix, dependancy_matrix, thresholds):
    xor_nodes = defaultdict(list)
    for current_node in event_indexes.values():
        xor_edges = []
        for other_node in event_indexes.values():
            xor_edges.append([other_node, current_node])
        above_threshold = edges_above_threshold(xor_edges, thresholds, succession_matrix, dependancy_matrix)
        for edge, above in above_threshold.items():
            if above:
                xor_nodes[current_node].append(edge[0])
    xor_nodes = { k: v for k, v in xor_nodes.items() if len(v) > 1 }
    print("Potential XOR joins (format -> 'end: [starts]'): {}".format(indexes_to_events(xor_nodes, event_indexes)))
    xor_filtered_nodes = xor_nodes.copy()
    for end_node, predecessors in xor_nodes.items():
        edges = []
        others = [end_node] + predecessors
        for predecessor in predecessors:
            for other in others:
                if predecessor == other:
                    continue
                edges.append([other, predecessor])
        below_threshold = edges_below_threshold(edges, thresholds, succession_matrix, dependancy_matrix)
        if not all(condition for condition in below_threshold.values()):
            xor_filtered_nodes.pop(end_node, None)

    print("Discovered XOR joins: {}".format(indexes_to_events(xor_filtered_nodes, event_indexes)))            
    return xor_filtered_nodes

In [11]:
events, logs = process_logs("TDlog.csv")
event_indexes, zero_matrix = zero_transition_matrix(events)
succession_matrix = generate_succession_matrix(events, logs)
dependancy_matrix = generate_dependancy_matrix(events, logs)

def find_and_splits(event_indexes, succession_matrix, dependancy_matrix, thresholds):
    and_nodes = defaultdict(list)
    for row in event_indexes.values():
        and_edges = []
        for col in event_indexes.values():
            and_edges.append([row, col])
        above_threshold = edges_above_threshold(and_edges, thresholds, succession_matrix, dependancy_matrix)
        for edge, above in above_threshold.items():
            if above:
                and_nodes[edge[0]].append(edge[1])
    and_nodes = { k: v for k, v in and_nodes.items() if len(v) > 1 }
    print("Potential AND splits (format -> 'start: [ends]'): {}".format(indexes_to_events(and_nodes, event_indexes)))    
    and_filtered_nodes = and_nodes.copy()
    frecuency_thresholds = thresholds.copy()
    frecuency_thresholds['dependancy'] = -100000000
    for start_node, successors in and_nodes.items():
        parallel_edges = []
        wrong_edges = []
        conditions = {}
        others = [start_node] + successors        
        for successor in successors:
            for other in others:
                if successor == other:
                    continue
                elif other in successors:
                    parallel_edges.append([successor, other])
                else:
                    wrong_edges.append([successor, other])
        above_frecuency_threshold = edges_above_frecuency_below_dependancy(parallel_edges, thresholds, succession_matrix, dependancy_matrix)                
        below_threshold = edges_above_threshold(wrong_edges, thresholds, succession_matrix, dependancy_matrix)                        
        if not all(condition for condition in below_threshold.values()):
            and_filtered_nodes.pop(start_node, None)
        if not all(condition for condition in above_frecuency_threshold.values()):
            and_filtered_nodes.pop(start_node, None)
    print("Discovered AND splits: {}".format(indexes_to_events(and_filtered_nodes, event_indexes)))
    return and_filtered_nodes

In [12]:
events, logs = process_logs("TDlog.csv")
event_indexes, zero_matrix = zero_transition_matrix(events)
succession_matrix = generate_succession_matrix(events, logs)
dependancy_matrix = generate_dependancy_matrix(events, logs)

def find_and_joins(event_indexes, succession_matrix, dependancy_matrix, thresholds):
    and_nodes = defaultdict(list)
    for current_node in event_indexes.values():
        and_edges = []
        for other_node in event_indexes.values():
            and_edges.append([other_node, current_node])
        above_threshold = edges_above_threshold(and_edges, thresholds, succession_matrix, dependancy_matrix)
        for edge, above in above_threshold.items():
            if above:
                and_nodes[current_node].append(edge[0])
    and_nodes = { k: v for k, v in and_nodes.items() if len(v) > 1 }
    print("Potential AND joins (format -> 'end: [starts]'): {}".format(indexes_to_events(and_nodes, event_indexes)))
    and_filtered_nodes = and_nodes.copy()
    for end_node, predecessors in and_nodes.items():
        parallel_edges = []
        wrong_edges = []
        conditions = {}
        others = [end_node] + predecessors        
        for predecessor in predecessors:
            for other in others:
                if predecessor == other:
                    continue
                elif other in predecessors:
                    parallel_edges.append([other, predecessor])
                else:
                    wrong_edges.append([other, predecessor])
        above_frecuency_threshold = edges_above_frecuency_below_dependancy(parallel_edges, thresholds, succession_matrix, dependancy_matrix)                
        below_threshold = edges_below_threshold(wrong_edges, thresholds, succession_matrix, dependancy_matrix)                        
        if not all(condition for condition in below_threshold.values()):
            and_filtered_nodes.pop(end_node, None)
        if not all(condition for condition in above_frecuency_threshold.values()):
            and_filtered_nodes.pop(end_node, None)
    print("Discovered AND joins: {}".format(indexes_to_events(and_filtered_nodes, event_indexes)))
    return and_filtered_nodes

In [13]:
events, logs = process_logs("TDlog.csv")
event_indexes, zero_matrix = zero_transition_matrix(events)
succession_matrix = generate_succession_matrix(events, logs)
dependancy_matrix = generate_dependancy_matrix(events, logs)

# Setted based on observation
thresholds = {"frecuency": 100, "dependancy": 0.2}  
print("Thresholds setted at:\n\t* Frecuency: {}\n\t* Dependancy: {}\n".format(thresholds['frecuency'], thresholds['dependancy']))

xor_split_nodes = find_xor_splits(event_indexes, succession_matrix, dependancy_matrix, thresholds)
xor_join_nodes = find_xor_joins(event_indexes, succession_matrix, dependancy_matrix, thresholds)
and_split_nodes = find_and_splits(event_indexes, succession_matrix, dependancy_matrix, thresholds)
and_join_nodes = find_and_joins(event_indexes, succession_matrix, dependancy_matrix, thresholds)

Thresholds setted at:
	* Frecuency: 100
	* Dependancy: 0.2

Potential XOR splits (format -> 'start: [ends]'): {'A': ['B', 'C'], 'C': ['F', 'G'], 'E': ['M', 'P'], 'F': ['H', 'I'], 'G': ['K', 'L']}
Discovered XOR splits: {'A': ['B', 'C'], 'C': ['F', 'G']}
Potential XOR joins (format -> 'end: [starts]'): {'J': ['H', 'I'], 'L': ['G', 'J', 'K', 'Q'], 'N': ['M', 'P']}
Discovered XOR joins: {}
Potential AND splits (format -> 'start: [ends]'): {'A': ['B', 'C'], 'C': ['F', 'G'], 'E': ['M', 'P'], 'F': ['H', 'I'], 'G': ['K', 'L']}
Discovered AND splits: {}
Potential AND joins (format -> 'end: [starts]'): {'J': ['H', 'I'], 'L': ['G', 'J', 'K', 'Q'], 'N': ['M', 'P']}
Discovered AND joins: {'J': ['H', 'I'], 'N': ['M', 'P']}


In [18]:
events, logs = process_logs("TDlog.csv")
def loops(events, logs):
    loops_finded = {}
    for log in logs.values():
        appearances = defaultdict(int)
        for event_index in range(len(log)):
            event = log[event_index]
            appearances[event] += 1
            if appearances[event] > 1:
                loops_finded[log[event_index-1]] = log[event_index]
    return loops_finded

loops(events, logs)
    


{'D': 'B', 'B': 'D'}